## STAT 842 Final Project Option 6 - Manim Code

### Prepare the Data
Note that the "stock_data.csv" file is needed to run the code.

In [1]:
import pandas as pd

def process_data(file, selected_symbols=None, num_months=3):
    # 1) Load and filter data
    df = pd.read_csv(file, parse_dates=["date"])
    df = df[df["symbol"].isin(selected_symbols)]

    # 2) Only keep the last 3 months data
    df["month"] = df["date"].dt.to_period("M")
    latest_months = df["month"].drop_duplicates().sort_values().iloc[-num_months:]
    df = df[df["month"].isin(latest_months)]
    df = df.sort_values(["symbol", "date"])

    # 3) Add direction column
    # Used to identify if the stock adjusted close price is 
    # greater than yesterday or not
    df["direction"] = df.groupby("symbol")["close"].diff().apply(
        lambda x: "UP" if x > 0 else "DOWN")

    # 4) Convert dates to numeric range (days since min date)
    min_date = df["date"].min()
    df["date_num"] = (df["date"] - min_date).dt.days.astype(float)

    return df, min_date

### Build the Manim Class

New parts that were not covered in class
1. A zigzag line
2. Shape - polygons and stars
3. Python pandas data processing

In [2]:
from manim import *

class StockManim(Scene):
    def construct(self):
        # Load the data
        selected_symbols = ["AAPL", "META", "TSLA", "MSFT"]
        df, min_date = process_data("stock_data.csv", selected_symbols = selected_symbols)
        
        # Make sure consistency with the poster
        self.camera.background_color = WHITE

        # Title
        title = Text("3-Month Stock Adjusted Close Prices", 
                     font_size=36, color=BLACK).to_edge(UP)
        # self.add(title)

        # Prepare different shapes to show the direction
        shape_map = {
            "AAPL": lambda c: Triangle(color=c, fill_color=c, fill_opacity=1).scale(0.2),
            "META": lambda c: Star(color=c, fill_color=c, fill_opacity=1).scale(0.2),
            "TSLA": lambda c: RegularPolygon(n=6, color=c, fill_color=c, fill_opacity=1).scale(0.2),
            # "AMZN": lambda c: Square(color=c, fill_color=c, fill_opacity=1).scale(0.2),
            "MSFT": lambda c: Circle(color=c, fill_color=c, fill_opacity=1).scale(0.2)
        }

        # Create Axes
        # - x_range goes from 0 to max of date_num
        # - y_range covers min -> max close price plus a little padding
        x_min = df["date_num"].min()
        x_max = df["date_num"].max()
        y_min = df["close"].min() - 80
        y_max = df["close"].max() + 100
        step_x = max(1, int((x_max - x_min) // 12))
        step_y = 50
        y_min = int(y_min)
        y_max = int(y_max)
        step_y = int(step_y)
        
        axes = Axes(
            x_range=[x_min, x_max, step_x],
            y_range=[y_min, y_max, step_y], 
            axis_config={"color": BLACK},
            y_axis_config={
                "include_numbers": True,
                "font_size": 26,
                # Exclude y_min from the labels:
                # Reason: add 0 and zigzag line
                "numbers_to_include": [
                    val
                    for val in range(y_min, y_max + 1, step_y)
                    if val != y_min
                ]
            }
        )
        # self.add(axes)
        
        # Get bottom of y-axis
        y_axis_start = axes.y_axis.get_start()
        
        # Add label 0 for y-axis
        label_0 = Text("0", font_size=20, color=BLACK)
        label_0.next_to(y_axis_start, buff = -0.5)
        self.add(label_0)

        # Zigzag line near the bottom of y-axis
        # Zigzag line since not start from 0
        zigzag = VMobject()
        zigzag.set_points_as_corners([
            y_axis_start ,
            y_axis_start + 0.15*LEFT + 0.08*UP,
            y_axis_start + 0.16 * UP,
            y_axis_start + 0.15*RIGHT + 0.24*UP,
            y_axis_start + 0.32 * UP
        ])
        zigzag.set_stroke(color=BLACK, width=3)
        # self.add(zigzag)

        # Add custom date labels on the x-axis
        # Label every Nth day
        # unique_dates = df[["date_num", "date"]].drop_duplicates().sort_values("date_num")
        # x_axis_labels = {}
        # # Label every 5th unique date
        # for i, row in unique_dates.iloc[::5].iterrows():
        #     day_value = row["date_num"]
        #     date_label_str = row["date"].strftime("%m-%d")  # "MM-DD"
        #     x_axis_labels[day_value] = Text(date_label_str, font_size=24, color=BLACK)
        
        tick_values = np.arange(x_min, x_max + 1, step_x)
        x_axis_labels = {}
        for tick in tick_values:
            # Find closest matching date
            closest_row = df.iloc[(df["date_num"] - tick).abs().argsort()[:1]]
            date_label_str = closest_row["date"].dt.strftime("%m-%d").values[0]
            x_axis_labels[tick] = Text(date_label_str, font_size=24, color=BLACK)

        axes.get_x_axis().add_labels(x_axis_labels)
        
        # Rotate X-axis labels
        for label in x_axis_labels.values():
            # Rotate each label ~30 degrees for visibility
            label.rotate(-30 * DEGREES) 
            label.shift(DOWN * 0.1)   
            label.scale(0.6)

        # Axis labels color and position
        x_label = axes.get_x_axis_label(
            Text("Date", color=BLACK, font_size=26),
            edge=DOWN,      
            direction=RIGHT*1.5+UP*0.14, 
            buff=5 # How far away from the axi
        )
  
        y_label = axes.get_y_axis_label(
            Text("Close Price", color=BLACK, font_size=26),
            edge=LEFT,      
            direction=UP*0.9,  
            buff=3.5     
        )
    
        y = axes.get_y_axis().numbers.set_color(BLACK)
        
        # self.add(x_label, y_label)
        
        # Animate the background settings at once
        self.play(
            Write(title),
            FadeIn(axes),
            Write(label_0),
            Write(x_label),
            Write(y_label),
            Create(zigzag), run_time=3.5
        )
        
        # Initilizaiton
        color_map = {
            "AAPL": "#1F77B4",
            "META": "#FF7F0E",
            "TSLA":"#F0E442",
            "MSFT": "#CC79A7"
        }
        
        lines = []
        for ticker in selected_symbols:
            # Pull out the sorted (date_num, close) points for this ticker
            ticker_df = df[df["symbol"] == ticker].sort_values("date_num")
            points = [
                axes.coords_to_point(row["date_num"], row["close"])
                for _, row in ticker_df.iterrows()
            ]
            # Create a line connecting these points
            line = VMobject()
            line.set_points_as_corners(points)
            
            # Choose a line color for each ticker
            line_color = color_map.get(ticker, WHITE)
            line.set_stroke(line_color, width=4)
            # self.add(line)
            # self.play(Create(line), run_time=2)
            lines.append(line)
        
        # Animate lines all at once
        # self.play(*[Create(l) for l in lines], run_time=2)

        # Plot shapes at (date_num, close)
        # Color shapes based on direction UP/DOWN
        shapes = []
        all_shapes = []
        for _, row in df.dropna(subset=["close"]).iterrows():
            x_val = row["date_num"]
            y_val = row["close"]
            ticker = row["symbol"]
            direction = row["direction"]
            
            # Set colors representing UP and DOWN
            color_choice = RED if direction == "UP" else GREEN
            shape = shape_map[ticker](color_choice).scale(0.3)
            shape.move_to(axes.coords_to_point(x_val, y_val))
            shapes.append(shape)
            
            # Fade each shape in
            # shape_anims.append(FadeIn(shape))
            # all_shapes.append(shape)
            
        # Legend title
        legend_title = Text("Price Direction", font_size=20, color=BLACK)

        # Red = Up
        red_marker = Dot(color=RED).scale(1.2)
        red_label = Text("Up", font_size=20, color=BLACK)
        red_group = VGroup(red_marker, red_label).arrange(RIGHT, buff=0.2)

        # Green = Down
        green_marker = Dot(color=GREEN).scale(1.2)
        green_label = Text("Down", font_size=20, color=BLACK)
        green_group = VGroup(green_marker, green_label).arrange(RIGHT, buff=0.2)

        # Combine all legend rows
        legend = VGroup(legend_title, red_group, green_group).arrange(DOWN, aligned_edge=LEFT, buff=0.3)

        # Position legend
        legend.to_corner(UR, buff=1)
        legend.shift(RIGHT * 0.9) 

        # self.play(FadeIn(legend))
        
        # Play shape animations in parallel
        # self.play(*shape_anims, run_time=2)
        self.play(
            *[Create(l) for l in lines], # Draw all lines and shapes
            *[FadeIn(s) for s in shapes],
            FadeIn(legend),
            run_time=8
        )

        # Label the company's name at the last point of each ticker
        label_anims = []
        labels = []
        for ticker in selected_symbols:
            last_row = df[df["symbol"] == ticker].iloc[-1]
            x_val = last_row["date_num"]
            y_val = last_row["close"]
            label_str = f"{ticker}"
            label = Text(label_str, font_size=24, color=BLUE)
            label.next_to(axes.coords_to_point(x_val, y_val), RIGHT, buff=0.2)
            labels.append(label)
            label_anims.append(FadeIn(label))
        
        self.play(*label_anims, run_time=2)
        self.wait(5)

%manim -qm -v WARNING StockManim

Manim Community v0.16.0.post0